In [ ]:
%load_ext autoreload
%autoreload 2
from scipy import sparse as sp
from matplotlib import pyplot as plt
import numpy as np

import sys, os
sys.path.append(os.getcwd() + "/08")
import aklt


In [ ]:
L = 14
J = 1
g = 1.5
H = aklt.gen_hamiltonian(L=L, g=g, J=J, periodic_boundary=False)
(E0,), psi_0 = sp.linalg.eigsh(H, k=1, which="SA")
psi_0 = psi_0.reshape(-1, 1)

# It's already normalized
assert np.allclose(np.dot(np.conj(psi_0).T, psi_0), 1)

print(f"{E0 = }")

In [ ]:
# Check if we can recreate the original state from MPS
M = aklt.get_MPS_tensor(psi_0, L, 2**(L//2))
print(f"{M.shape = }")
assert np.allclose(M.ravel(), psi_0.ravel())
del M

In [ ]:
# Find maximum compression ratio
# "Lossless" as defined by np.allclose
# Man, do while would be nice right now
def find_compression_ratio(psi: np.ndarray, L: int, chi_0: int = 20) -> None:
    M = aklt.get_MPS_tensor(psi, L, 2**(L//2))
    chi = chi_0
    while np.allclose(M.ravel(), psi.ravel()):
        chi -= 1
        M = aklt.get_MPS_tensor(psi, L, chi)
    
    if chi == chi_0:
        print("MPS representation did not correspond to original representation "
              f"with {chi_0 = }.")
        return

    # Found the first chi where we have compression losses
    chi += 1
    print(f"{chi = }")
    Ms_full_size = sum([M.size for M in aklt.get_MPS_list(psi, L, 2**(L//2))])
    Ms_comp_size = sum([M.size for M in aklt.get_MPS_list(psi, L, chi)])

    compression_ratio = Ms_full_size / Ms_comp_size
    print(f"{compression_ratio = :.2f}")

In [ ]:
print("Ground state: ")
find_compression_ratio(psi_0, L, 13)

In [ ]:
print("Random state: ")
psi_rnd = np.random.normal(size=(2**L)) + 1j * np.random.normal(size=(2**L))
psi_rnd /= np.dot(np.conj(psi_rnd).T, psi_rnd)
find_compression_ratio(psi_rnd, L, 2**(L//2))
del psi_rnd

In [ ]:
psi_exact = aklt.get_MPS_list(psi_0, L, 2**(L//2) + 1)
psi_compr = aklt.get_MPS_list(psi_0, L, 10)

In [ ]:
print(f"{aklt.overlap(psi_exact, psi_exact) = }")
print(f"{aklt.overlap(psi_compr, psi_compr) = }")
print(f"{aklt.overlap(psi_exact, psi_compr) = }")

In [ ]:
psi_up = np.zeros(2**L, dtype=np.complex128)
psi_up[0] = 1
MPS_up = aklt.get_MPS_list(psi_up, L, 2**(L//2) + 1)

print(f"{aklt.overlap(psi_exact, MPS_up) = :.3f}")

In [ ]:
print("GHZ state, we expect chi = 2")
GHZ = np.zeros(2**L, dtype=np.complex128)
GHZ[0] = 2**-0.5 + 0j
GHZ[-1] = 2**-0.5 + 0j
find_compression_ratio(GHZ, L, 5)